In [1]:
import os
import itertools
import numpy as np
import pandas as pd
from tqdm import tqdm, tqdm_notebook
from ph_likes.util import queries
from ph_likes.util import settings
from ph_likes.post_similarity.cosine_post_similarity import CosinePostSimilarity

/home/can/anaconda3/lib/python3.6/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [2]:
# Load model
model = CosinePostSimilarity()

In [6]:
df_post = queries.get_post_table()
df_postlike = queries.get_postlike_table()

In [7]:
YEAR = 2018
MONTH = 11
df_post_ = df_post[(df_post['featured_at'].dt.year == YEAR) & (df_post['featured_at'].dt.month == MONTH)]
df_postlike_ = df_postlike[df_postlike['post_id'].isin(df_post_['id'].unique())]

In [8]:
liked_posts = df_postlike_[df_postlike_['user_id'] == settings.EXAMPLE_USER_ID]['post_id'].unique()
not_liked_posts = np.array(list(set(df_post_['id'].unique()) - set(liked_posts)))
print(len(liked_posts))
print(len(not_liked_posts))

5
727


In [9]:
avg_similarity_for_posts = dict()
for post_id in tqdm_notebook(not_liked_posts):
    avg_similarity = np.mean([model.get_similarity(post_id, i) for i in liked_posts])
    avg_similarity_for_posts[post_id] = avg_similarity

In [10]:
df_avg_similarity_for_posts = pd.DataFrame(pd.Series(avg_similarity_for_posts)).reset_index()
df_avg_similarity_for_posts.columns = ['id', 'avg_similarity']
df_avg_similarity_for_posts = df_avg_similarity_for_posts.sort_values(by='avg_similarity', ascending=False)

In [11]:
df_avg_similarity_for_posts = df_avg_similarity_for_posts.merge(df_post_, on='id')

In [12]:
df_avg_similarity_for_posts.to_csv(os.path.join(settings.LOCAL_DATA_PATH, str(settings.EXAMPLE_USER_ID) + '_recs.csv'),
                                   sep='\t',
                                   index=False)